In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from easyquery import Query, QueryMaker

from astropy.table import Table, join, vstack, unique
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.wcs import WCS

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import *
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import *
print(SAGA.__version__)

In [ ]:
import warnings
from astropy.wcs import FITSFixedWarning
from astropy.io.fits.verify import VerifyWarning
warnings.filterwarnings("ignore", category=FITSFixedWarning)
warnings.filterwarnings("ignore", category=VerifyWarning)

from matplotlib.patches import Ellipse

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
host = saga.host_catalog.load_single("NGC7541")

base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = QueryMaker.equals("HOSTID", host["HOSTID"]).filter(base)

host_rvir = np.rad2deg(np.arcsin(0.3 / host["DIST"]))
host_ra_corr = np.abs(np.cos(np.deg2rad(host["DEC"])))
host_center = (host["RA"], host["DEC"])
host_name = host["COMMON_NAME"]
fp = fits.open("/home/yymao/Documents/Research/SAGA/PaperII/host-dss-images/{}.fits".format(host["HOSTID"]), output_verify="silentfix")

host_background = fp[0].data.copy()
host_background_header = fp[0].header
del fp[0].data, fp[0]
fp.close()

In [ ]:
ncols = 4
nrows = 1

fig = plt.figure(figsize=(14, 14*nrows/ncols*1.05))
gs = plt.GridSpec(nrows, ncols, fig)

for i, opt in enumerate([
    dict(q=Query(C.basic_cut2), s=7, lw=0, c="C8", label="All galaxies $r_o < 20.75$"),
    dict(q=Query(C.basic_cut2, C.paper1_targeting_cut), s=7, lw=0, c="C0", label="Conservative color cuts\n(Geha et al. 2017)"),
    dict(q=Query(C.basic_cut2, C.paper2_targeting_cut) | C.is_sat, s=7, lw=0, c="C2", label="Improved Color/SB cuts\n(Mao et al. 2020)"),
    dict(q=Query(C.is_sat), s=28, lw=1.5, c='none', edgecolors="C6", label="Satellites"),
]):
    ax = plt.subplot(gs[i], projection=WCS(host_background_header), label=host["HOSTID"])
    ax.imshow(host_background, cmap='Greys')
    t = opt["q"].filter(base)
    del opt["q"]
    ax.scatter(t["RA"], t["DEC"], transform=ax.get_transform('fk5'), **opt)
    ax.add_patch(Ellipse((host["RA"], host["DEC"]), 2.0*host_rvir/host_ra_corr, 2.0*host_rvir, 
                 edgecolor='#999999', facecolor='none', transform=ax.get_transform('fk5'))) 

    ax.set_xlim(-0.5, host_background.data.shape[1] - 0.5)
    ax.set_ylim(-0.5, host_background.data.shape[0] - 0.5)
    for c in ax.coords:
        c.set_ticklabel(exclude_overlapping=True, size=9, pad=0)
        c.display_minor_ticks(True)
        c.set_axislabel('', size=0)
        
    ax.text(0.05, 1.05, opt["label"], transform=ax.transAxes, va='bottom', fontsize=15)